In [1]:
import numpy as np
import pandas as pd

import random

from sklearn.datasets import make_classification

In [2]:
X, y = make_classification(n_samples=50, n_features=20, n_informative=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [3]:
display(X.head())
print(X.shape)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13
0,0.212170,0.588157,0.373931,-1.213137,-0.431857,-1.805413,0.374652,-0.328778,1.662872,0.682909,-0.635783,-0.096290,-0.706476,1.475155
1,-2.136309,-0.340340,-1.518135,2.791709,-0.348785,-0.697299,-3.616860,-0.644757,-3.150791,0.153453,-1.975852,1.927038,-0.225723,1.335919
2,-2.995246,0.418912,-1.147293,4.705204,0.109306,-0.134241,0.297226,2.162918,-6.801806,1.573732,0.234367,-0.348181,-3.033989,-2.326364
3,-4.434309,1.959684,0.313601,0.497666,0.864826,2.565846,-1.654235,-1.603219,1.411960,-0.621943,-2.532930,-0.387911,0.313242,4.148565
4,-6.564804,0.302972,0.394640,1.189341,-2.472628,4.819816,-1.126806,-1.453735,-2.691496,-0.259630,-1.587911,-0.205920,-0.589160,0.759591


(1000, 14)


In [4]:
display(y.head())
print(y.shape)

0    0
1    0
2    0
3    1
4    1
dtype: int32

(1000,)


In [5]:
class MyTreeClf():

    def __init__(self, max_depth = 5, min_samples_split = 2, max_leafs = 20):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs


    def __repr__(self):
        return f'MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}'

In [6]:
tree = MyTreeClf()
print(tree)

MyTreeClf class: max_depth=5, min_samples_split=2, max_leafs=20


In [72]:
def entropy(y):
    return - np.sum([(i/len(y))*np.log2(i/len(y)) if i>0 else 0 for i in y.value_counts(sort=False)])

def get_best_split(X, y):
    X, y = X.copy(), y.copy()
    S0 = entropy(y)
    max_ig = 0
    for col in X:
        values = sorted(X[col].unique())
        dividers = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]
        for divider in dividers:
            split_right = y.iloc[X[X[col] > divider].index]
            split_right_counts = split_right.value_counts(sort=False)
            split_left = y.iloc[X[X[col] <= divider].index]
            split_left_counts = split_left.value_counts(sort=False)
            ig = S0 - (len(split_right)/len(y))*entropy(split_right) - (len(split_left)/len(y))*entropy(split_left)
            if ig > max_ig:
                col_name, split_value, max_ig = col, divider, ig
    return col_name, split_value, max_ig

In [73]:
get_best_split(X, y)

('col_4', -0.032813398968507546, 0.113919270120095)

In [74]:
y.iloc[X[X.col_0 > 0.2].index].value_counts(sort=False)

0    154
1    181
Name: count, dtype: int64